In [1]:
import pandas as pd
import selenium.webdriver as wd
from selenium.webdriver.common.by import By
import time

In [69]:
op = wd.EdgeOptions()
op.add_extension('ublock.crx')

In [333]:
try:
    dr.quit()
except:
    pass

In [3]:
dr = wd.Edge(options=op)

In [ ]:
urls = []
names = []
prices = []
rats = []
numrevs = []
asins = []

url = 'https://www.amazon.in/s?k=bags&page={}&crid=2M096C61O4MLT&qid=1690021477'

for page in range(16, 31):
    
    dr.get(url.format(page))

    time.sleep(5)
    
    cls = '.sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16'.replace(' ', '.')

    eles = [el for el in dr.find_elements(By.CSS_SELECTOR, cls) if el.is_displayed()]
    
    for el in eles:
        if 'out of' in el.find_element(By.CSS_SELECTOR, '.a-section.a-spacing-none.a-spacing-top-micro').get_attribute('innerText'):
            uniq = 'a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal'
            txt = el.find_element(By.CSS_SELECTOR, '.a-section.a-spacing-none.a-spacing-top-micro').get_attribute('innerText')
            urls.append(el.find_element(By.CSS_SELECTOR, uniq).get_attribute('href'))
            names.append(el.find_element(By.CSS_SELECTOR, uniq).text)
            rats.append(txt.split('\n')[0])
            numrevs.append(eval(txt.split('\n')[1].replace(',' ,'')))
            prices.append(eval(el.find_element(By.CLASS_NAME, 'a-price-whole').text.replace(',' ,'')))
            asins.append(el.get_attribute('data-asin'))

    print(page, end='\r')

In [383]:
df = pd.DataFrame(dict(name=names, url=urls, rating = rats, number_of_ratings = numrevs, price=prices, asin=asins))

In [384]:
df.to_csv('URLS.csv', index=None)

# Manufacturer and Description

In [70]:
dr.quit()

In [71]:
dr = wd.Edge(options=op)

In [72]:
mans = []
dets = []
prod = []

for i, url in enumerate(df['url']):

    dr.get(url)

    time.sleep(2)
    
    dr.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    if dr.find_elements(By.ID, 'prodDetails'):
        txt = dr.find_element(By.ID, 'prodDetails').text
        man = txt[txt.find('Manufacturer')+13:].split('\n')[0]
    
    elif dr.find_elements(By.ID, 'detailBullets_feature_div'):
        txt = dr.find_element(By.ID, 'detailBullets_feature_div').text
        man = txt[txt.find('Manufacturer')+14:].split('\n')[0].strip()
    
    mans.append(man)
    dets.append(txt)
    prod.append(dr.find_elements(By.CSS_SELECTOR, '#feature-bullets, #featurebullets_feature_div')[0].text)

    del man
    del txt

    print('{:%}'.format((i+1)/300), end='\r')

In [73]:
df['Manufacturer'] = mans
df['Product Details'] = prod
df['Details'] = dets

In [74]:
df.to_csv('final_output.csv', index=None)

In [78]:
dr.quit()